<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np

In [104]:
!git clone https://github.com/emanhamed/Houses-dataset

fatal: destination path 'Houses-dataset' already exists and is not an empty directory.


In [105]:
df=pd.read_csv("Houses-dataset/Houses Dataset/HousesInfo.txt",header=None,delim_whitespace=True,
               names=["bedrooms","bathrooms","size","zipcode","price"])

In [106]:
type(df)

pandas.core.frame.DataFrame

In [107]:
def cleanData(df):
    # compute the number of entries per zipcode
    zipcodes=df['zipcode'].value_counts().keys().tolist()
    counts=df['zipcode'].value_counts().tolist()
    #discard all zipcodes ocurring less than 20 times
    for count,zipcode in zip(counts,zipcodes):
      if count<20:
        idx=df[df['zipcode']==zipcode].index
        df.drop(idx,inplace=True)
    return df

In [108]:
dataset=cleanData(df)

In [109]:
type(dataset)

pandas.core.frame.DataFrame

In [110]:
dataset['price'].max()

5858000

In [6]:
import shutil
from_prefix="Houses-dataset/Houses Dataset/"
to_prefix="images/"
suffix="_frontal.jpg"
!mkdir -p images
for newidx,oldidx in enumerate(df.index.tolist()):
  oldname=from_prefix+str(oldidx)+suffix
  newname=to_prefix+str(newidx)+suffix
  #print("moving from %s to %s"%(oldname,newname))
  shutil.copy(oldname,newname)

In [7]:
dataset.to_csv("cleansedDataset.csv",index=False)

In [96]:
dataset['price']

ValueError: ignored

In [111]:
class CustomDataset(Dataset):
  def __init__(self,csvFile,imgDir):
    dataset=pd.read_csv(csvFile)
    dummy=pd.get_dummies(dataset['zipcode'])
    price=dataset['price']
    self.max_price=price.max()
    df=dataset.drop(['price','zipcode'],axis=1)
    self.data=pd.concat([dataset,dummy,price],axis=1)
  def __len__(self):
    return len(self.data)

  def __getitem__(self,idx):
    #
    return self.data.iloc[idx,:-1].to_numpy(dtype=np.float32),np.float32(self.data.iloc[idx,-1])/max_price

In [65]:
dataset=CustomDataset("cleansedDataset.csv","images")

In [66]:
itr=iter(dataset)
x=next(itr)


In [83]:
train_loader=DataLoader(dataset,batch_size=32,shuffle=True)

In [84]:
itr=iter(train_loader)

In [85]:
x=next(itr)

In [86]:
len(x)

2

In [87]:
x[1].size()

torch.Size([32])

In [88]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.relu=nn.ReLU()
    self.fc1=nn.Linear(in_features=13,out_features=32)
    self.fc2=nn.Linear(in_features=32,out_features=16)
    self.fc3=nn.Linear(in_features=16,out_features=1)
  def forward(self,x):
    x=self.fc1(x)
    x=self.relu(x)
    x=self.fc2(x)
    x=self.relu(x)
    x=self.fc3(x)
    return x

In [92]:
model=Net()
x[0].size()

torch.Size([32, 13])

In [94]:
from torch.optim import SGD
from torch.nn import MSELoss


In [95]:
model=Net()
optimizer=SGD(model.parameters(),lr=0.001,momentum=0.9)
loss_fn=MSELoss()
epochs=10
for epoch in range(epochs):
  for input,price in train_loader:
    output=model(input)
    optimizer.zero_grad()
    loss=loss_fn(output.squeeze(),price)
    loss.backward()
    optimizer.step()
    if(epoch==0):
      print(loss)
  



tensor(1.4028e+12, grad_fn=<MseLossBackward0>)
tensor(inf, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
tensor(nan, grad_fn=<MseLossBackward0>)
